# Funciones de Usuario (udfs)

* 30 min | Última modificación: Noviembre 07, 2019

Apache Pig permite la ejecución de funciones escritas en Java, Jython (http://www.jython.org/index.html), JavaScript, Ruby, Groovy y Python. Información detallada de la ejecución de udfs en estos lenguajes puede ser obtenida en https://pig.apache.org/docs/latest/udf.html#python-udfs.


Véase http://www.jython.org/index.html. Para su uso, el script de Python debe estar ubicado en el directorio de trabajo. Para poder usar Jython, hay dos opciones; en la primera se debe indicar la ubicación del inteprete en la variable `PIG_CLASSPATH`. En algunos casos es necesario establacer la variable `PYTHONPATH` para que Pig pueda compilar las udfs. Para mayor información revise https://pig.apache.org/docs/latest/udf.html#jython-udfs.

(deprecated) En la segunda se puede usar `org.apache.pig.scripting.jython.JythonScriptEngin` como intérprete.

Cell magic `%%pig`
---

In [1]:
from IPython.core.magic import Magics, cell_magic, magics_class
from pexpect import spawn

TIMEOUT = 60
PROG = 'pig'
PROMPT = ["\r\n>> ", "\r\ngrunt> "]
DISCARD = ['INFO  org.apache', 'WARN  org.apache']

@magics_class
class Magic(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.app = spawn(PROG, timeout=60)
        self.app.expect(PROMPT)

    @cell_magic
    def pig(self, line, cell):
        cell_lines = [cell_line.strip() for cell_line in cell.split("\n")]
        cell_lines = [cell_line for cell_line in cell_lines if cell_line != ""]
        for cell_line in cell_lines:
            self.app.sendline(cell_line)
            self.app.expect(PROMPT, timeout=TIMEOUT)
            output = self.app.before.decode()
            output = output.replace('\r\n', '\n')
            output = output.split('\n')
            output = [output_line.strip() for output_line in output]
            for output_line in output:
                if output_line not in cell_lines:
                    if not any(word in output_line for word in DISCARD):
                        print(output_line)
        return None


def load_ipython_extension(ip):
    ip.register_magics(Magic(ip))


load_ipython_extension(ip=get_ipython())

In [1]:
%load_ext bigdata

In [2]:
%pig_start

In [3]:
%timeout 300

In [2]:
%%writefile udf.tsv
A	B	1
C	D	2
E	F	3
G	H	4
I	J	5

Writing udf.tsv


In [5]:
!hadoop fs -put udf.tsv

In [3]:
%%writefile pigudf.py

#from pig_util import outputSchema

@outputSchema("as:int")
def square(num):
    if num == None:
        return None
    return ((num) * (num))

@outputSchema("word:chararray")
def concatenar(word1, word2):
    return (word1 + word2)

Writing pigudf.py


In [4]:
%%pig 
--
-- registra las funciones en el script `pigudf.py`.
--
REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
REGISTER 'pigudf.py' using jython as myudf;

u = LOAD 'udf.tsv' USING PigStorage() 
    AS (c1:CHARARRAY, 
        c2:CHARARRAY, 
        c3:INT);

b = FOREACH u GENERATE myudf.concatenar(c1, c2), myudf.square(c3);
DUMP b;

(AB,1)
(CD,4)
(EF,9)
(GH,16)
(IJ,25)


**Piggy Bank**

Son funciones de usuario escritas en Java por la comunidad. 

---

In [5]:
!rm pigudf.py pig_*  udf.tsv